# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df.copy()

def changeHomeOwn(home_ownership):
    if home_ownership == "RENT":
        return 1
    elif home_ownership == "OWN":
        return 2
    else:
        return 0

X["home_ownership"] = X["home_ownership"].apply(changeHomeOwn)

def changeVerification(verification_status):
    if verification_status == "Source Verified":
        return 1
    elif verification_status == "Verified":
        return 2
    else:
        return 0

X["verification_status"] = X["verification_status"].apply(changeVerification)

def changeApplication(application_type):
    if application_type == "Individual":
        return 1
    elif application_type == "Joint App":
        return 2
    else:
        return 0

X["application_type"] = X["application_type"].apply(changeApplication)

def changeListStatus(initial_list_status):
    if initial_list_status == "f":
        return 1
    elif initial_list_status == "w":
        return 2
    else:
        return 0

X["initial_list_status"] = X["initial_list_status"].apply(changeListStatus)

def changeHardship(hardship_flag):
    if hardship_flag == "N":
        return 1
    else:
        return 0

X["hardship_flag"] = X["hardship_flag"].apply(changeHardship)

def changeDebtSettlement(debt_settlement_flag):
    if debt_settlement_flag == "N":
        return 1
    else:
        return 0

X["debt_settlement_flag"] = X["debt_settlement_flag"].apply(changeDebtSettlement)

X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,1,66000.0,1,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,1,1
1,25000.0,0.2000,929.09,0,105000.0,2,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,1,1
2,20000.0,0.2000,529.88,0,56000.0,2,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,1,1
3,10000.0,0.1640,353.55,1,92000.0,2,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,1,1
4,22000.0,0.1474,520.39,0,52000.0,0,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,1,1


In [6]:
X.drop(["issue_d", "loan_status", "pymnt_plan", "next_pymnt_d"], axis=1, inplace=True)
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,1,66000.0,1,27.24,0.0,0.0,8.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,1,1
1,25000.0,0.2000,929.09,0,105000.0,2,20.23,0.0,0.0,17.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,1,1
2,20000.0,0.2000,529.88,0,56000.0,2,24.26,0.0,0.0,8.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,1,1
3,10000.0,0.1640,353.55,1,92000.0,2,31.44,0.0,1.0,10.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,1,1
4,22000.0,0.1474,520.39,0,52000.0,0,18.76,0.0,1.0,14.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,1,1


In [7]:
# Create our target
y = df["loan_status"].ravel()
y[:5]

array(['low_risk', 'low_risk', 'low_risk', 'low_risk', 'low_risk'],
      dtype=object)

In [8]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,0.571152,8.821371e+04,0.669994,21.778153,0.217766,0.497697,12.587340,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,1.0,1.0
std,10277.348590,0.048130,288.062432,0.677080,1.155800e+05,0.719105,20.199244,0.718367,0.758122,6.022869,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.000000,0.000000,2.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,1.0,1.0
25%,9000.000000,0.088100,265.730000,0.000000,5.000000e+04,0.000000,13.890000,0.000000,0.000000,8.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,1.0,1.0
50%,15000.000000,0.118000,404.560000,0.000000,7.300000e+04,1.000000,19.760000,0.000000,0.000000,11.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.000000,1.040000e+05,1.000000,26.660000,0.000000,1.000000,16.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,1.0,1.0
max,40000.000000,0.308400,1676.230000,2.000000,8.797500e+06,2.000000,999.000000,18.000000,5.000000,72.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,1.0,1.0


In [9]:
# Check the balance of our target values
# YOUR CODE HERE

In [10]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [11]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [13]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [15]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.7222327705174634

In [16]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   61,    40],
       [ 2728, 14376]], dtype=int64)

In [17]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.60      0.84      0.04      0.71      0.50       101
   low_risk       1.00      0.84      0.60      0.91      0.71      0.52     17104

avg / total       0.99      0.84      0.61      0.91      0.71      0.52     17205



In [18]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_

sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.07243128582205462, 'total_rec_prncp'),
 (0.06167215676927323, 'last_pymnt_amnt'),
 (0.05976764418653361, 'total_rec_int'),
 (0.05646987994694546, 'total_pymnt_inv'),
 (0.05612731915845889, 'total_pymnt'),
 (0.033927821588950885, 'int_rate'),
 (0.01920704335312487, 'mths_since_recent_inq'),
 (0.018327668069454437, 'dti'),
 (0.01824834587077127, 'max_bal_bc'),
 (0.01718566548316008, 'installment'),
 (0.016470448443789934, 'annual_inc'),
 (0.01640130594469689, 'out_prncp_inv'),
 (0.016289185328867747, 'mths_since_rcnt_il'),
 (0.01625928137372472, 'revol_bal'),
 (0.015817114248135394, 'out_prncp'),
 (0.015721373888566925, 'total_bc_limit'),
 (0.015400325185121964, 'bc_util'),
 (0.015327674865864237, 'tot_hi_cred_lim'),
 (0.01523827514810909, 'mo_sin_old_rev_tl_op'),
 (0.015036475073607575, 'mo_sin_old_il_acct'),
 (0.014765091818450267, 'avg_cur_bal'),
 (0.014713095561890228, 'bc_open_to_buy'),
 (0.014633503970195274, 'total_bal_il'),
 (0.01460359881675219, 'il_util'),
 (0.0145165371760

### Easy Ensemble Classifier

In [19]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [20]:
# Calculated the balanced accuracy score
y_pred2 = eec.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred2)

0.7217482564439793

In [21]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred2)

array([[   68,    33],
       [ 3930, 13174]], dtype=int64)

In [22]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.67      0.77      0.03      0.72      0.51       101
   low_risk       1.00      0.77      0.67      0.87      0.72      0.52     17104

avg / total       0.99      0.77      0.67      0.86      0.72      0.52     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The model that had the better balanced accuracy score is the Balanced Random Forest Classifier with 0.72223 , compared to Easy Ensemble Classifier with 0.72175.

2. Which model had the best recall score?

    The model that had the better recall score is the  Balanced Random Forest Classifier with 0.84, compared to 0.77 of the Easy Ensemble Classifier.

3. Which model had the best geometric mean score?

    The model that had the better geometric mean score is the Easy Ensemble Classifier with 0.72, compared to 0.71 of the Balances Random Forest Classifier.

4. What are the top three features?

    The top three features are the Total Received Principal, Last Payment Amount and Total Received Interest.